In [ ]:
# basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# image manipulation
from PIL import Image

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="PA7iGNwB8gUmOWCyfNt3")
project = rf.workspace().project("pill-pick")
model = project.version(2).model

In [ ]:
image_dir = os.path.join(os.path.expanduser('~'), 'Downloads', 'images')
image_cropped_dir = os.path.join(os.path.expanduser('~'), 'Downloads', 'images_cropped')
counter = 0

for image_name in os.listdir(image_dir):
    
    if image_name.endswith('.jpg') or image_name.endswith('.png'):
        image_path = os.path.join(image_dir, image_name)
        result = model.predict(image_path, confidence=40, overlap=30).json()
        image = np.array(Image.open(image_path))
    
        # Extracting best pill per image:
        if len(result['predictions']) > 0:
            x = int(result['predictions'][0]['x'])
            y = int(result['predictions'][0]['y'])
            x_size = int(result['predictions'][0]['width'])
            y_size = int(result['predictions'][0]['height'])
            
            image_cropped = image[y-y_size//2:y+y_size//2,x-x_size//2:x+x_size//2,:]
            image_cropped=Image.fromarray(image_cropped)
            image_cropped.save(os.path.join(image_cropped_dir, image_name))
            print(f"✅ {image_name} cropped and saved to {image_cropped_dir}.")
            
        else:
            image=Image.fromarray(image)
            image.save(os.path.join(image_cropped_dir, image_name))
            print(f"❌ {image_name} not cropped but saved to {image_cropped_dir}.")
        counter += 1
        

print(f"\n✅ All images cropped and saved! Total: {counter}")